In [2]:
# Config
import os

# Basic
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Monitoring
from tqdm.notebook import tqdm

# IO
from os.path import join, exists, basename, dirname, splitext, expanduser
from glob import glob

# Parallel processing
from joblib import Parallel, delayed
import re
from PIL import Image
import supervision as sv
from supervision.metrics import MeanAveragePrecision
from supervision.metrics import MetricTarget
from tempfile import mkdtemp

from ultralytics import YOLO


In [4]:
home_dir="/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/"
base_state = "haryana" ## state for which model is trained
target_state = "bihar" ## state for which model is to be validated
test_state = "bihar" ## state for which model is to be validatd during training
model_name = "yolo11m-obb.pt" ## trained model name
task="obb" ## task for which model is trained
conf=0.001 ## confidence threshold
iou=0.33 ## iou threshold
ratio="same_class_count_10_120_1000_4x"

# base_dir="/home/patel_zeel/kiln_compass_24/data/"

gt_image_dir= f"../data/swinir_data/test_{target_state}_{ratio}/images"
gt_label_dir=f"../data/swinir_data/test_{target_state}_{ratio}/labels"
# /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/cyclegan_data/test_output/fake_test_bihar_same_class_count_10_120_1000
# gt_image_dir= "../data/cyclegan_data/test_output/fake_test_bihar_same_class_count_10_120_1000/"
# gt_label_dir= "../data/cyclegan_data/test_output/fake_bihar_same_class_count_10_120_1000/labels"
print(gt_image_dir)
print(gt_label_dir)

## class information in yml file 
data_yml_save_path=mkdtemp()
data_yml = f"""train: dummy
val: dummy
nc: 3
names: ["CFCBK", "FCBK", "Zigzag"]
"""
data_yml_path = f"{data_yml_save_path}/data.yml"
with open(data_yml_path, "w") as f:
    f.write(data_yml)

print(data_yml_path)  


class_count="10_120_1000"

# "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/train_bihar_test_haryana_0.25_obb_v1_yolo11m-obb.pt_640_128_300
# model_path="/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/train_haryana_test_haryana_0.25_obb_v1_yolo11m-obb.pt_640_128_300/weights/best.pt"
## model path
yolo_runs_dir = "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/"
# /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/train_fake_bihar_test_bihar_same_class_count_10_120_1000_obb_v1_yolo11m-obb.pt_640_128_500
# model_dir= f"{yolo_runs_dir}train_{base_state}_test_{test_state}_same_class_count_{class_count}_{task}_v1_{model_name}_640_128_500/weights/"
# model_path = f"{model_dir}best.pt"
# print(model_path)
model_path="../runs/obb_sr/train_haryana_same_class_count_10_120_1000_4x_test_bihar_same_class_count_10_120_1000_4x_10_120_1000_4x_obb_v1_yolo11m-obb.pt_2560_8_300/weights/best.pt"
## output path
result_dir=f"{yolo_runs_dir}train_{base_state}_test_{target_state}_{class_count}_{task}_v2_{model_name}_640_128_300_{conf}_{iou}/"
# prediction_dir = join(result_dir,"labels")
# # print(result_dir)
# print(prediction_dir)


../data/swinir_data/test_bihar_same_class_count_10_120_1000_4x/images
../data/swinir_data/test_bihar_same_class_count_10_120_1000_4x/labels
/tmp/tmp8r_to6y8/data.yml


In [5]:
## Supervision dataset
sv_dataset=sv.DetectionDataset.from_yolo(gt_image_dir,gt_label_dir,data_yml_path)
print(sv_dataset)
print(len(sv_dataset))
sv_dataset.classes

## Yolo model
model = YOLO(model_path)

687


In [6]:
targets=[]
predictions=[]

for name,_,gt_detection in tqdm(sv_dataset):
    file_name=splitext(basename(name))[0]
    # prediction_path=join(prediction_dir,f"{file_name}.txt")
    ultralytics_result=model(name,imgsz=2560,iou=0.33,conf=0.001,exist_ok=True,save_txt=True,max_det=300,name=result_dir, verbose=False)[0]
    sv_detection=sv.Detections.from_ultralytics(ultralytics_result)
    targets.append(gt_detection)
    predictions.append(sv_detection)









  0%|          | 0/687 [00:00<?, ?it/s]

Results saved to /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/train_haryana_test_bihar_10_120_1000_obb_v2_yolo11m-obb.pt_640_128_300_0.001_0.33
1 label saved to /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/train_haryana_test_bihar_10_120_1000_obb_v2_yolo11m-obb.pt_640_128_300_0.001_0.33/labels
Results saved to /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/train_haryana_test_bihar_10_120_1000_obb_v2_yolo11m-obb.pt_640_128_300_0.001_0.33
2 labels saved to /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/train_haryana_test_bihar_10_120_1000_obb_v2_yolo11m-obb.pt_640_128_300_0.001_0.33/labels
Results saved to /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/train_haryana_test_bihar_10_120_1000_obb_v2_yolo11m-obb.pt_640_128_300_0.001_0.33
3 labels saved to /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/train_haryana_test_bihar_10_120_1000_obb_v2_yolo11m-obb.pt_640_128_300

In [7]:
## mAP calculation 
mAP_metric = MeanAveragePrecision(class_agnostic=False)

mAP_result=mAP_metric.update(predictions,targets).compute()
class_wise_mAP=mAP_result.ap_per_class[:,0].tolist()
# print(f"mAP_result: {mAP_result}")
matched_classes=mAP_result.matched_classes.tolist()
print(f"Matched classes: {matched_classes}")
num_classes=3
final_class_wise_mAP = [0]*num_classes

for cls, mAP in zip(matched_classes, class_wise_mAP):
    print(f"cls: {cls}, mAP: {mAP}")
    final_class_wise_mAP[cls] = mAP


print(f"class_wise_mAP: {final_class_wise_mAP}")


mAP_metric = MeanAveragePrecision(class_agnostic=True)
mAP_result = mAP_metric.update(predictions, targets).compute()
class_agnostic_result = mAP_result.ap_per_class[:, 0].tolist()
# print(f"mAP_result: {mAP_result}")
print(f"class_wise_mAP: {class_wise_mAP}")
print(f"class_agnostic_result: {class_agnostic_result}")

Matched classes: [0, 1, 2]
cls: 0, mAP: 0.0
cls: 1, mAP: 0.01416628921905103
cls: 2, mAP: 0.41925698616561097
class_wise_mAP: [0.0, 0.01416628921905103, 0.41925698616561097]
class_wise_mAP: [0.0, 0.01416628921905103, 0.41925698616561097]
class_agnostic_result: [0.4211653597824483]


In [8]:
target_state = "swinir_bihar_4x_300"
base_state = "haryana" 

In [9]:
index = pd.MultiIndex.from_tuples([], names=["Base State", "class_count", "Target State"])
result_df = pd.DataFrame(columns=["CFCBK", "FCBK", "Zigzag", "mAP", "Class-agnostic AP"], index=index)
result_df.loc[(base_state, class_count, target_state), :] = final_class_wise_mAP + [mAP_result.mAP_scores[0]] + class_agnostic_result
display(result_df)


,,,CFCBK,FCBK,Zigzag,mAP,Class-agnostic AP
Base State,class_count,Target State,,,,,
haryana,10_120_1000,swinir_bihar_4x_300,0.0,0.014166,0.419257,0.421165,0.421165


In [34]:
output_csv_path = f"{home_dir}result_csv/different_state_to_bihar_{ratio}_v4_mAP_results.csv"


In [35]:
# output_csv_path = f"{home_dir}result_csv/different_state_to_{target_state}_mAP_results.csv"
os.makedirs(dirname(output_csv_path), exist_ok=True)


if not os.path.exists(output_csv_path):
    result_df.to_csv(output_csv_path, header=True)
    print(f"Results saved to {output_csv_path}")
else:
    result_df.to_csv(output_csv_path, mode='a', header=False)
    print(f"Results appended to {output_csv_path}")

Results appended to /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/result_csv/different_state_to_bihar_same_class_count_10_120_1000_4x_v4_mAP_results.csv


In [1]:
base_state_2 =  ## state for which model is trained
output_csv_path_2 = f"{home_dir}result_csv/{base_state_2}_to_different_state_mAP_results.csv"

result_west_bengal=pd.read_csv(output_csv_path)
result_punjab=pd.read_csv(output_csv_path_2)
result=pd.concat([result_west_bengal,result_punjab],ignore_index=True)
display(result)

SyntaxError: invalid syntax (1003467514.py, line 1)

In [48]:
csv_path=f"../result_csv/different_state_to_bihar_same_class_count_10_120_1000_v4_mAP_results.csv"
result=pd.read_csv(csv_path)
# sort by mAP
result=result.sort_values(by='mAP',ascending=False)
display(result.style.background_gradient(cmap='viridis',axis=0))

,Base State,class_count,Target State,CFCBK,FCBK,Zigzag,mAP,Class-agnostic AP
1,bihar,10_120_1000,bihar,0.399586,0.016424,0.607663,0.594605,0.594605
5,bihar,5_60_500,bihar,0.108911,0.006941,0.577577,0.587014,0.587014
4,bihar,15_240_1500,bihar,0.624077,0.010945,0.592665,0.569439,0.569439
2,swinir_bihar,10_120_1000,swinir_bihar,0.434435,0.010334,0.568917,0.565028,0.565028
0,swinir_bihar,10_120_1000,bihar,0.441749,0.020593,0.563421,0.552524,0.552524
6,punjab,10_120_1000,bihar,0.000000,0.003102,0.510558,0.506449,0.506449
7,west_bengal,10_120_1000,bihar,0.291726,0.009801,0.478794,0.502281,0.502281
8,west_bengal,15_240_1500,bihar,0.458384,0.032326,0.000000,0.391730,0.391730
3,bihar,10_120_1000,swinir_bihar,0.289958,0.000437,0.287359,0.287281,0.287281


In [29]:
def apply_state_color(val):
    if val == 'West Bengal':
        return 'background-color: yellow'  # color for West Bengal (Yellow)
    elif val == 'Punjab':
        return 'background-color: green'  # color for Punjab (Green)
    else:
        return ''  # no color if it's other

# Apply the color styling for 'Base State' and 'Target State' columns
styled_result = result.style\
    .applymap(apply_state_color, subset=["Base State", "Target State"])\
    .background_gradient(subset=["CFCBK", "FCBK", "Zigzag", "mAP", "Class-agnostic AP"], cmap="coolwarm")\
    .set_properties(**{"text-align": "center"})\
    .set_table_styles([
        {"selector": "th", "props": [("font-weight", "bold"), ("text-align", "center")]},
        {"selector": "tr:nth-child(even)", "props": [("background-color", "#f9f9f9")]}
    ])

# Displaying the styled DataFrame
display(styled_result)

,Base State,Target State,CFCBK,FCBK,Zigzag,mAP,Class-agnostic AP
0,west_bengal,punjab,0.000000,0.019683,0.425855,0.492828,0.492828
1,west_bengal,uttar_pradesh,0.000000,0.129238,0.197902,0.417703,0.417703
2,west_bengal,haryana,0.000000,0.004189,0.236194,0.270739,0.270739
3,west_bengal,bihar,0.000000,0.033873,0.412915,0.451084,0.451084
4,punjab,haryana,0.000000,0.008151,0.643177,0.659907,0.659907
5,punjab,bihar,0.000000,0.012281,0.338602,0.322256,0.322256
6,punjab,uttar_pradesh,0.000000,0.035343,0.137244,0.215012,0.215012
7,punjab,west_bengal,0.000000,0.016273,0.133746,0.199013,0.199013
